In [1]:
import os
from glob import glob

import numpy as np

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models

import keras
from keras.models import Model
from keras.layers import Add, Activation, Concatenate, Conv2D, Dropout
from keras.layers import Flatten, Input, GlobalAveragePooling2D, MaxPooling2D
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0Metal device set to: Apple M1 Pro



2023-03-25 20:43:21.569419: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-25 20:43:21.569533: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
#Set the image size as per the VGG16 archetecture
IMAGE_SIZE = [256, 256]

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# !ls '/content/drive'

In [6]:
#Give dataset path
train_path = 'data/grayscale/train'
# validation_path = 'X:\\Projects\\tomato\\val'
test_path = 'data/grayscale/test'

In [7]:
def count(directory, counter=0):
    """returns number of files in dir and subdirs"""
    for pack in os.walk(directory):
        for f in pack[2]: counter += 1
    return directory + " : " + str(counter) + " files"

In [8]:
print('total images for training :', count(train_path))
# print('total images for validation :', count(validation_path))
print('total images for validation :', count(test_path))

total images for training : data/grayscale/train : 14478 files
total images for validation : data/grayscale/test : 3694 files


In [9]:
folders = glob('data/grayscale/train/*')
print(len(folders))

10


In [10]:
BATCH_SIZE = 32

train_generator = ImageDataGenerator(preprocessing_function = preprocess_input, validation_split = 0.1) #VGG16 preprocessing

# validation_generator = ImageDataGenerator(preprocessing_function=preprocess_input) 

test_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
# VGG16 preprocessin

In [11]:
class_subset = sorted(os.listdir('data/grayscale/train'))

print(class_subset)

traingen = train_generator.flow_from_directory(train_path,
                                               target_size=(256, 256),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=5)

validgen = train_generator.flow_from_directory(train_path,
                                               target_size=(256, 256),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=4)

testgen = test_generator.flow_from_directory(test_path,
                                             target_size=(256, 256),
                                             class_mode=None,
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=4)

['bacterialspot', 'earlyblight', 'healthy', 'lateblight', 'leafmold', 'mosaicvirus', 'septorialeafspot', 'spidermites', 'targetspot', 'yellowleafcurlvirus']
Found 13026 images belonging to 10 classes.
Found 1442 images belonging to 10 classes.
Found 3692 images belonging to 10 classes.


# **VGG16**

In [12]:
model1 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

2023-03-24 23:21:15.962934: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-24 23:21:15.962958: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
for layer in model1.layers:
  layer.trainable = False

In [14]:
model1.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [15]:
flatten_layer = layers.Flatten()
batnorm = layers.BatchNormalization()
dropout = layers.Dropout(0.20)
prediction_layer = layers.Dense(10, activation='sigmoid')


vgg16 = models.Sequential([
    model1,
    flatten_layer,
    batnorm,
    dropout,
    prediction_layer
])

In [16]:
vgg16.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 batch_normalization (BatchN  (None, 32768)            131072    
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 10)                327690    
                                                                 
Total params: 15,173,450
Trainable params: 393,226
Non-trainable params: 14,780,224
______________________________________

In [17]:
optimizer=Adam(learning_rate=0.001)
vgg16.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


from keras.callbacks import ModelCheckpoint, EarlyStopping
optim_1 = Adam(learning_rate=0.001)
#n_classes=3

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 50

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='data/grayscale/vgg16.h5',
                                  save_best_only=True,
                                  monitor='val_accuracy',
                                  mode='max',
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')


vgg_history = vgg16.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[tl_checkpoint_1], 
                            verbose=1)

Epoch 1/50


2023-03-24 21:12:21.465183: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-24 21:12:21.691702: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


407/407 [==============================] - ETA: 0s - loss: 0.7739 - accuracy: 0.8263

2023-03-24 21:14:30.599616: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_accuracy improved from -inf to 0.87847, saving model to data/grayscale/vgg16.h5
407/407 [==============================] - 143s 350ms/step - loss: 0.7739 - accuracy: 0.8263 - val_loss: 0.7891 - val_accuracy: 0.8785
Epoch 2/50
407/407 [==============================] - ETA: 0s - loss: 0.1902 - accuracy: 0.9558
Epoch 2: val_accuracy improved from 0.87847 to 0.89306, saving model to data/grayscale/vgg16.h5
407/407 [==============================] - 140s 343ms/step - loss: 0.1902 - accuracy: 0.9558 - val_loss: 0.8212 - val_accuracy: 0.8931
Epoch 3/50
407/407 [==============================] - ETA: 0s - loss: 0.1423 - accuracy: 0.9686
Epoch 3: val_accuracy improved from 0.89306 to 0.89444, saving model to data/grayscale/vgg16.h5
407/407 [==============================] - 138s 340ms/step - loss: 0.1423 - accuracy: 0.9686 - val_loss: 1.0206 - val_accuracy: 0.8944
Epoch 4/50
407/407 [==============================] - ETA: 0s - loss: 0.1150 - accuracy: 0.9739
Epoch 4: val_accuracy

In [17]:
# Generate predictions
vgg16.load_weights('data/grayscale/vgg16.h5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

vgg_preds = vgg16.predict(testgen)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)

2023-03-24 23:21:29.372665: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-24 23:21:29.422939: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3692/3692 [==============================] - 44s 12ms/step


In [18]:
vgg_acc = accuracy_score(true_classes, vgg_pred_classes)
print("VGG16 Model Accuracy without Fine-Tuning: {:.2f}%".format(vgg_acc * 100))

VGG16 Model Accuracy without Fine-Tuning: 34.78%


In [19]:
print(classification_report(true_classes,vgg_pred_classes))

              precision    recall  f1-score   support

           0       0.32      1.00      0.48       423
           1       0.14      0.47      0.21       208
           2       0.61      0.83      0.70       319
           3       0.35      0.33      0.34       403
           4       0.17      0.23      0.19       190
           5       0.23      0.14      0.17        79
           6       0.34      0.14      0.20       356
           7       0.53      0.34      0.42       338
           8       0.61      0.10      0.17       285
           9       1.00      0.11      0.20      1091

    accuracy                           0.35      3692
   macro avg       0.43      0.37      0.31      3692
weighted avg       0.57      0.35      0.31      3692



In [20]:
print(confusion_matrix(true_classes,vgg_pred_classes))

[[421   1   0   1   0   0   0   0   0   0]
 [105  97   1   2   2   0   1   0   0   0]
 [ 28  23 266   1   1   0   0   0   0   0]
 [132 123  13 134   0   0   1   0   0   0]
 [ 40  59   8  40  43   0   0   0   0   0]
 [  5  20  15   7  21  11   0   0   0   0]
 [126 110  14  31  19   6  50   0   0   0]
 [ 30  40  59  19  25  10  39 116   0   0]
 [ 54  57  57  20   9   3  32  25  28   0]
 [390 187   3 123 133  18  23  78  18 118]]


# **ResNet152**

In [38]:
model6 = tf.keras.applications.ResNet152(include_top=False, weights='imagenet', input_shape=(256,256,3))

for layer in model6.layers:
    layer.trainable = False

In [39]:
model6.summary()

Model: "resnet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

In [40]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
batnorm = layers.BatchNormalization()
dropout = layers.Dropout(0.20)
prediction_layer = layers.Dense(10, activation='sigmoid')

resnet152 = models.Sequential([
    model6,
    flatten_layer,
    batnorm,
    dropout,
    prediction_layer
])

In [41]:
resnet152.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152 (Functional)       (None, 8, 8, 2048)        58370944  
_________________________________________________________________
flatten_4 (Flatten)          (None, 131072)            0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 131072)            524288    
_________________________________________________________________
dropout_4 (Dropout)          (None, 131072)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1310730   
Total params: 60,205,962
Trainable params: 1,572,874
Non-trainable params: 58,633,088
_________________________________________________________________


In [42]:
optimizer=Adam(learning_rate=0.001)
resnet152.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


from keras.callbacks import ModelCheckpoint, EarlyStopping
optim_1 = Adam(learning_rate=0.001)
#n_classes=3

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 50

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='X:\\Projects\\grayscale\\resnet152.h5',
                                  save_best_only=True,
                                  monitor='val_accuracy',
                                  mode='max',
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')


vgg_history = resnet152.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[tl_checkpoint_1], 
                            verbose=1)

Epoch 1/50
209/209 [==============================] - 126s 558ms/step - loss: 2.6781 - accuracy: 0.7706 - val_loss: 2.0787 - val_accuracy: 0.8410

Epoch 00001: val_accuracy improved from -inf to 0.84103, saving model to X:\Projects\grayscale\resnet152.h5


C:\Users\bhala\anaconda3\envs\NNDL\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/50
209/209 [==============================] - 117s 557ms/step - loss: 0.7556 - accuracy: 0.9343 - val_loss: 2.1941 - val_accuracy: 0.8573

Epoch 00002: val_accuracy improved from 0.84103 to 0.85734, saving model to X:\Projects\grayscale\resnet152.h5
Epoch 3/50
209/209 [==============================] - 117s 561ms/step - loss: 0.3652 - accuracy: 0.9653 - val_loss: 2.6918 - val_accuracy: 0.8845

Epoch 00003: val_accuracy improved from 0.85734 to 0.88451, saving model to X:\Projects\grayscale\resnet152.h5
Epoch 4/50
209/209 [==============================] - 118s 565ms/step - loss: 0.2765 - accuracy: 0.9750 - val_loss: 2.7069 - val_accuracy: 0.8709

Epoch 00004: val_accuracy did not improve from 0.88451
Epoch 5/50
209/209 [==============================] - 120s 576ms/step - loss: 0.3093 - accuracy: 0.9747 - val_loss: 3.6652 - val_accuracy: 0.8682

Epoch 00005: val_accuracy did not improve from 0.88451
Epoch 6/50
209/209 [==============================] - 121s 577ms/step - loss: 0.

209/209 [==============================] - 137s 656ms/step - loss: 0.1070 - accuracy: 0.9955 - val_loss: 8.2981 - val_accuracy: 0.8791

Epoch 00041: val_accuracy did not improve from 0.90217
Epoch 42/50
209/209 [==============================] - 155s 743ms/step - loss: 0.1684 - accuracy: 0.9936 - val_loss: 7.9788 - val_accuracy: 0.8872

Epoch 00042: val_accuracy did not improve from 0.90217
Epoch 43/50
209/209 [==============================] - 154s 737ms/step - loss: 0.2024 - accuracy: 0.9934 - val_loss: 7.4715 - val_accuracy: 0.8913

Epoch 00043: val_accuracy did not improve from 0.90217
Epoch 44/50
209/209 [==============================] - 154s 735ms/step - loss: 0.1273 - accuracy: 0.9955 - val_loss: 7.9350 - val_accuracy: 0.8845

Epoch 00044: val_accuracy did not improve from 0.90217
Epoch 45/50
209/209 [==============================] - 153s 730ms/step - loss: 0.1788 - accuracy: 0.9940 - val_loss: 8.0859 - val_accuracy: 0.8791

Epoch 00045: val_accuracy did not improve from 0.902

In [47]:
import numpy as np
# Generate predictions
resnet152= tf.keras.models.load_model('X:\\Projects\\grayscale\\resnet152.h5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

resnet152_preds = resnet152.predict(testgen)
resnet152_pred_classes = np.argmax(resnet152_preds, axis=1)

In [48]:
from sklearn.metrics import accuracy_score

resnet152_acc = accuracy_score(true_classes, resnet152_pred_classes)
print("ResNet152 Model Accuracy without Fine-Tuning: {:.2f}%".format(resnet152_acc * 100))

ResNet152 Model Accuracy without Fine-Tuning: 33.23%


In [49]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(true_classes,resnet152_pred_classes))

              precision    recall  f1-score   support

           0       0.34      0.99      0.51       200
           1       0.20      0.57      0.29       200
           2       0.58      0.76      0.66       200
           3       0.29      0.23      0.25       200
           4       0.36      0.20      0.26       191
           5       0.36      0.31      0.33        75
           6       0.29      0.08      0.13       200
           7       0.55      0.12      0.20       200
           8       0.33      0.03      0.05       200
           9       1.00      0.01      0.03       200

    accuracy                           0.33      1866
   macro avg       0.43      0.33      0.27      1866
weighted avg       0.44      0.33      0.27      1866



In [ ]:
confusion_matrix(true_classes,resnet152_pred_classes)

## SqueezeNet

In [12]:
def SqueezeNet_11(input_shape, nb_classes, dropout_rate=None, compression=1.0):
    """
    Creating a SqueezeNet of version 1.1

    2.4x less computation over SqueezeNet 1.0 implemented above.

    Arguments:
        input_shape  : shape of the input images e.g. (224,224,3)
        nb_classes   : number of classes
        dropout_rate : defines the dropout rate that is accomplished after last fire module (default: None)
        compression  : reduce the number of feature-maps

    Returns:
        Model        : Keras model instance
    """

    input_img = Input(shape=input_shape)

    x = Conv2D(int(64*compression), (3,3), activation='relu', strides=(2,2), padding='same', name='conv1')(input_img)

    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool1')(x)

    x = create_fire_module(x, int(16*compression), name='fire2')
    x = create_fire_module(x, int(16*compression), name='fire3')

    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool3')(x)

    x = create_fire_module(x, int(32*compression), name='fire4')
    x = create_fire_module(x, int(32*compression), name='fire5')

    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool5')(x)

    x = create_fire_module(x, int(48*compression), name='fire6')
    x = create_fire_module(x, int(48*compression), name='fire7')
    x = create_fire_module(x, int(64*compression), name='fire8')
    x = create_fire_module(x, int(64*compression), name='fire9')

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # Creating last conv10
    x = output(x, nb_classes)

    return Model(inputs=input_img, outputs=x)


In [13]:
def output(x, nb_classes):
    x = Conv2D(nb_classes, (1,1), strides=(1,1), padding='valid', name='conv10')(x)
    x = GlobalAveragePooling2D(name='avgpool10')(x)
    x = Activation("softmax", name='softmax')(x)
    return x

In [14]:
def create_fire_module(x, nb_squeeze_filter, name, use_bypass=False):
    """
    Creates a fire module

    Arguments:
        x                 : input
        nb_squeeze_filter : number of filters of squeeze. The filtersize of expand is 4 times of squeeze
        use_bypass        : if True then a bypass will be added
        name              : name of module e.g. fire123

    Returns:
        x                 : returns a fire module
    """

    nb_expand_filter = 4 * nb_squeeze_filter
    squeeze    = Conv2D(nb_squeeze_filter,(1,1), activation='relu', padding='same', name='%s_squeeze'%name)(x)
    expand_1x1 = Conv2D(nb_expand_filter, (1,1), activation='relu', padding='same', name='%s_expand_1x1'%name)(squeeze)
    expand_3x3 = Conv2D(nb_expand_filter, (3,3), activation='relu', padding='same', name='%s_expand_3x3'%name)(squeeze)

    axis = get_axis()
    x_ret = Concatenate(axis=axis, name='%s_concatenate'%name)([expand_1x1, expand_3x3])

    if use_bypass:
        x_ret = Add(name='%s_concatenate_bypass'%name)([x_ret, x])

    return x_ret


In [15]:
def get_axis():
    axis = -1 if K.image_data_format() == 'channels_last' else 1
    return axis

In [16]:
model_11 = SqueezeNet_11(input_shape=(224,224,3), nb_classes=1000)

2023-03-25 20:43:44.821691: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-25 20:43:44.821719: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [17]:
model_11.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 maxpool1 (MaxPooling2D)        (None, 55, 55, 64)   0           ['conv1[0][0]']                  
                                                                                              

In [18]:
flatten_layer = layers.Flatten()
batnorm = layers.BatchNormalization()
dropout = layers.Dropout(0.20)
prediction_layer = layers.Dense(10, activation='sigmoid')


SqueezeNet = models.Sequential([
    model_11,
    flatten_layer,
    batnorm,
    dropout,
    prediction_layer
])

In [19]:
SqueezeNet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 1000)              1235496   
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 batch_normalization (BatchN  (None, 1000)             4000      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 10)                10010     
                                                                 
Total params: 1,249,506
Trainable params: 1,247,506
Non-trainable params: 2,000
__________________________________________

In [ ]:
optimizer=Adam(learning_rate=0.001)
SqueezeNet.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


from keras.callbacks import ModelCheckpoint, EarlyStopping
optim_1 = Adam(learning_rate=0.001)
#n_classes=3

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 50

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='data/grayscale/squeezenet.h5',
                                  save_best_only=True,
                                  monitor='val_accuracy',
                                  mode='max',
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')


squeezenet_history = SqueezeNet.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[tl_checkpoint_1],
                            verbose=1)

Epoch 1/50


2023-03-25 20:43:49.122694: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-25 20:43:49.582736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


407/407 [==============================] - ETA: 0s - loss: 1.6392 - accuracy: 0.4103

2023-03-25 20:44:28.267917: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_accuracy improved from -inf to 0.34097, saving model to data/grayscale/squeezenet.h5
407/407 [==============================] - 42s 99ms/step - loss: 1.6392 - accuracy: 0.4103 - val_loss: 1.7531 - val_accuracy: 0.3410
Epoch 2/50
407/407 [==============================] - ETA: 0s - loss: 1.3019 - accuracy: 0.5422
Epoch 2: val_accuracy improved from 0.34097 to 0.59514, saving model to data/grayscale/squeezenet.h5
407/407 [==============================] - 39s 97ms/step - loss: 1.3019 - accuracy: 0.5422 - val_loss: 1.2578 - val_accuracy: 0.5951
Epoch 3/50
407/407 [==============================] - ETA: 0s - loss: 1.0706 - accuracy: 0.6291
Epoch 3: val_accuracy improved from 0.59514 to 0.61667, saving model to data/grayscale/squeezenet.h5
407/407 [==============================] - 39s 96ms/step - loss: 1.0706 - accuracy: 0.6291 - val_loss: 1.0890 - val_accuracy: 0.6167
Epoch 4/50
407/407 [==============================] - ETA: 0s - loss: 0.9424 - accuracy: 0.6781
Epoch 4: val

In [21]:
# Generate predictions
SqueezeNet.load_weights('data/grayscale/squeezenet.h5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

squeezenet_preds = SqueezeNet.predict(testgen)
squeezenet_pred_classes = np.argmax(squeezenet_preds, axis=1)

2023-03-25 21:19:24.054140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3692/3692 [==============================] - 24s 6ms/step


In [22]:
squeezenet_acc = accuracy_score(true_classes, squeezenet_pred_classes)
print("SqueezeNet Model Accuracy without Fine-Tuning: {:.2f}%".format(squeezenet_acc * 100))

SqueezeNet Model Accuracy without Fine-Tuning: 84.67%


In [23]:
print(classification_report(true_classes,squeezenet_pred_classes))

              precision    recall  f1-score   support

           0       0.87      0.92      0.89       423
           1       0.68      0.52      0.59       208
           2       0.96      0.96      0.96       319
           3       0.76      0.77      0.76       403
           4       0.73      0.77      0.75       190
           5       0.92      0.58      0.71        79
           6       0.81      0.77      0.79       356
           7       0.86      0.74      0.80       338
           8       0.74      0.79      0.76       285
           9       0.91      0.98      0.95      1091

    accuracy                           0.85      3692
   macro avg       0.82      0.78      0.80      3692
weighted avg       0.84      0.85      0.84      3692



In [24]:
print(confusion_matrix(true_classes,squeezenet_pred_classes))

[[ 388    3    1    7    3    0    6    0    0   15]
 [  17  109    0   34    8    0   10    2    3   25]
 [   0    0  307    0    1    0    1    1    9    0]
 [   6   23    2  312   12    0   22    0    8   18]
 [   2    3    0   23  146    1    5    4    0    6]
 [   0    1    0    0   10   46    6    4    3    9]
 [  17   11    4   25   12    2  273    0   10    2]
 [   1    1    2    4    5    1    6  250   42   26]
 [   6    6    4    7    1    0    8   26  224    3]
 [   9    3    0    1    1    0    1    3    2 1071]]
